In [1]:
import pandas as pd
from os import getcwd, listdir
from os.path import join, isfile, exists, abspath, pardir
from sys import path
import re

##### Configs

In [2]:
database = "sdm"
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")
path.append(scripts_dir)

In [3]:
from connect import Neo4jConnection
from utils import run_query_with_result

In [4]:
driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

##### Helper methods

In [5]:
def run_query(query, **kwargs):
    """
    Basic wrapper around 'driver' object
    """
    return run_query_with_result(driver, query, **kwargs)

#### Queries

In [6]:
def most_cited_papers():
    query = """
        Match (d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper' and toInteger(d.cited_count) > 0
        With j.name as conference, d.title as paper, sum(toInteger(d.cited_count)) as cited_count
        order by cited_count desc
        return conference, collect(paper)[..3] as papers, collect(cited_count)[..3] as total_cited_count
    """
    return run_query(query=query)

In [7]:
most_cited_papers()

,conference,papers,total_cited_count
0,Proceedings of SPIE - The International Societ...,[Near-to-eye electroholography via guided-wave...,"[165, 45, 30]"
1,Proceedings of the ASME Design Engineering Tec...,[Kinematics and discretization of curved-fold ...,"[65, 52, 39]"
2,Proceedings of the ASME Turbo Expo,[Predicting efficiency of a turbine driven by ...,"[54, 24, 18]"
3,Proceedings of the American Control Conference,[A Class of Logistic Functions for Approximati...,"[24, 12, 8]"
4,Optics InfoBase Conference Papers,[3D hydrodynamic focusing for optofluidics usi...,"[18, 9]"
...,...,...,...
89,Proceedings of the 20th International Conferen...,[Using instagram to increase student engagemen...,"[1, 1]"
90,"2018 Aviation Technology, Integration, and Ope...","[Exploration of three dimensional, hierarchica...",[1]
91,Proceedings of the International Masonry Socie...,[Strut-and-tie models for masonry walls with o...,[1]
92,"Wind Energy Symposium, 2018",[Benefits of two turbine rotor diameters and h...,[1]


In [8]:
def community():
    query = """
        MATCH (a:Author)<-[p:written_by]-(d:Document)-[r:published_in]->(j:Journal)
        Where d.document_type='Conference Paper'
        with j.name as conference, a.author_id as author_id, a.name as author_name, count(p) as count_paper
        order by author_name
        where count_paper > 3
        return conference, collect(author_name) as authors, sum(count_paper) as total_count
    """
    return run_query(query=query)

In [9]:
community()

,conference,authors,total_count
0,Optics InfoBase Conference Papers,"[ Adams M.R., Arashida Y., Ashida M., Atkin...",711
1,Proceedings of the International Telemetering ...,"[ Afran M.S., Hogstrom C., Josephson C., Na...",96
2,Proceedings of the ASME Design Engineering Tec...,"[ Anderson M., Avila A., Bai R., Bowden A.E...",1638
3,Proceedings of SPIE - The International Societ...,"[ Antshel K.M., Avval T.G., Bakker G., Bear...",1635
4,Geotechnical Special Publication,"[ Ayoubi P., Bender C., Besendorfer J., Bey...",420
5,Proceedings of the International Joint Confere...,"[ Baliyarasimhuni S.P., Butail S., Crandall ...",64
6,SAE Technical Papers,"[ Basko R., Carlson B., Hopman A., Hovanski...",64
7,"ASEE Annual Conference and Exposition, Confere...","[ Bateman T.C., Davies R., Retherford J., S...",114
8,Proceedings of the American Control Conference,"[ Beard R.W., Bidstrup C., Johnson C., John...",96
9,"Minerals, Metals and Materials Series","[ Beyerlein I.J., Carsley J.E., Field D.P., ...",56
